In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from datetime import timedelta, datetime
#import holidays

import matplotlib
import pickle
import seaborn as sns

import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objs as go

matplotlib.pyplot.style.use('ggplot')
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.6f' % x)
pd.set_option('display.width', 500)

In [2]:
uf_test = pd.read_csv('../../data/user_features_test.csv')
users_test = pd.read_csv('../../data/users_test.csv')

In [3]:
df_test = pd.read_csv('../../data/df_test_preprocessed.csv')

In [4]:
df_test = pd.merge(df_test, users_test, on='ID')

In [5]:
df_test.drop(columns=['first_open_date','first_open_timestamp','local_first_open_timestamp'], inplace=True)

In [6]:
country_cols =   ['United States',
                  'Brazil',        
                  'Mexico',        
                  'Argentina',     
                  'France',        
                  'Germany',       
                  'India',         
                  'Colombia',      
                  'Spain',         
                  'Italy',         
                  'Chile',         
                  'Turkey',        
                  'Ecuador',       
                  'Peru',          
                  'Indonesia',     
                  'United Kingdom',
                  'Canada',        
                  'Venezuela']

In [7]:
df_test['country'] = df_test['country'].apply(lambda x: x if x in country_cols else 'Other')

In [8]:
brand_cols = ['Apple',
              'Samsung',            
              'Xiaomi',             
              'Motorola',           
              'OPPO',               
              'Vivo',               
              'Huawei',             
              'Honor',              
              'Real']

In [9]:
df_test['device_brand'] = df_test['device_brand'].apply(lambda x: x if x in brand_cols else 'Other')

In [10]:
df_test.drop(columns=['device_model'], inplace=True)

In [11]:
df_test['ad_network'] = df_test['ad_network'].apply(lambda x: x if x not in ['restricted', 'ironsource_int', 'Cross_sale'] else 'Other')

In [12]:
df_test['ad_network'] = np.where(df_test['ad_network'].isnull(), 'Nan', df_test['ad_network'])

In [13]:
df_test.head()

,ID,open_year,open_month,first_prediction,log_first_prediction,economic_class,Retention_count_D0_3,Retention_count_D4_7,Retention_count_D8_11,Retention_count_D12_15,Avg_Lvl_Count_D0_3,Avg_Lvl_Count_D4_7,Avg_Lvl_Count_D8_11,Avg_Lvl_Count_D12_15,Avg_Duration_Lvl0_3,Avg_Duration_Lvl4_7,Avg_Duration_Lvl8_10,Total_AdRevenueD0_3,Total_AdRevenueD4_7,Total_AdRevenueD8_11,Total_AdRevenueD12_15,Total_IAPRevenueD0_3,Total_IAPRevenueD4_7,Total_IAPRevenueD8_11,Total_IAPRevenueD12_15,Total_AdRevenue,Total_IAPRevenue,country,platform,device_category,device_brand,has_ios_att_permission,ad_network
0,878594,2024,5,1.444805,0.367975,lower_middle_class,4,1,0,0,4.000000,0.000000,0.000000,0.000000,81.333333,164.750000,116.333333,0.012716,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012716,0.000000,Argentina,Android,mobile,Motorola,False,applovin_int
1,878595,2024,1,9.147972,2.213532,lower_middle_class,1,0,0,0,0.750000,0.000000,0.000000,0.000000,103.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Mexico,Android,mobile,Other,False,applovin_int
2,878596,2024,2,40.731158,3.706993,upper_middle_class,1,0,0,0,0.750000,0.000000,0.000000,0.000000,74.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,France,Android,mobile,Motorola,False,applovin_int
3,878597,2024,2,4.967959,1.603009,lower_middle_class,2,2,2,0,0.750000,4.250000,0.500000,0.000000,508.000000,695.750000,394.333333,0.000000,0.156159,0.112909,0.000000,0.000000,0.000000,0.000000,0.000000,0.269068,0.000000,Brazil,Android,mobile,Samsung,False,applovin_int
4,878598,2024,5,2.445842,0.894389,lower_middle_class,3,2,0,0,11.000000,8.750000,0.000000,0.000000,56.333333,85.250000,55.000000,0.004202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004202,0.000000,Peru,Android,mobile,Xiaomi,False,applovin_int


In [13]:
cols =  ['ID', 'first_prediction', 'log_first_prediction', 'economic_class', 
         'Retention_count_D0_3', 'Retention_count_D4_7', 'Retention_count_D8_11', 'Retention_count_D12_15',
         'Avg_Lvl_Count_D0_3', 'Avg_Lvl_Count_D4_7', 'Avg_Lvl_Count_D8_11', 'Avg_Lvl_Count_D12_15',
         'Avg_Duration_Lvl0_3', 'Avg_Duration_Lvl4_7', 'Avg_Duration_Lvl8_10', 
         'Total_IAPRevenue', 'Total_AdRevenue',
        'country', 'platform', 'device_category', 'device_brand', 'has_ios_att_permission', 'ad_network']

In [14]:
df_test = df_test[cols]

In [15]:
df_nan = df_test[df_test['economic_class'] == 'Nan']

In [16]:
df_test = df_test[df_test['economic_class'] != 'Nan']

**df_nan**

In [17]:
df_copy = df_nan.copy()

In [18]:
country_dummies = pd.get_dummies(df_copy.country , prefix = 'country', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, country_dummies], axis = 1)

platform_dummies = pd.get_dummies(df_copy.platform , prefix = 'platform', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, platform_dummies], axis = 1)

device_category_dummies = pd.get_dummies(df_copy.device_category , prefix = 'device_category', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, device_category_dummies], axis = 1)

device_brand_dummies = pd.get_dummies(df_copy.device_brand , prefix = 'device_brand', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, device_brand_dummies], axis = 1)

has_ios_att_permission_dummies = pd.get_dummies(df_copy.has_ios_att_permission , prefix = 'has_ios_att_permission', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, has_ios_att_permission_dummies], axis = 1)

ad_network_dummies = pd.get_dummies(df_copy.ad_network , prefix = 'ad_network', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, ad_network_dummies], axis = 1)

In [19]:
df_copy.drop(columns=['economic_class', 'country', 'platform', 'device_category', 'device_brand',
                     'has_ios_att_permission', 'ad_network'], inplace=True)

In [20]:
df_nan = df_copy.copy()

In [21]:
del(df_copy)

In [24]:
df_nan.head()

,ID,first_prediction,log_first_prediction,Retention_count_D0_3,Retention_count_D4_7,Retention_count_D8_11,Retention_count_D12_15,Avg_Lvl_Count_D0_3,Avg_Lvl_Count_D4_7,Avg_Lvl_Count_D8_11,Avg_Lvl_Count_D12_15,Avg_Duration_Lvl0_3,Avg_Duration_Lvl4_7,Avg_Duration_Lvl8_10,Total_IAPRevenue,Total_AdRevenue,country_Brazil,country_Canada,country_Chile,country_Colombia,country_Ecuador,country_France,country_Germany,country_India,country_Indonesia,country_Italy,country_Mexico,country_Other,country_Peru,country_Spain,country_Turkey,country_United Kingdom,country_United States,country_Venezuela,platform_iOS,device_category_tablet,device_brand_Honor,device_brand_Huawei,device_brand_Motorola,device_brand_OPPO,device_brand_Other,device_brand_Samsung,device_brand_Vivo,device_brand_Xiaomi,has_ios_att_permission_True,ad_network_Nan,ad_network_Other,ad_network_applovin_int,ad_network_googleadwords_int,ad_network_unityads_int
7,878601,NaN,NaN,1,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
65,878659,NaN,NaN,1,0,0,0,0.750000,0.000000,0.000000,0.000000,76.666667,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
86,878680,NaN,NaN,1,0,0,0,3.000000,0.000000,0.000000,0.000000,64.000000,115.000000,131.333333,0.000000,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
116,878710,NaN,NaN,1,1,0,0,1.000000,0.000000,0.000000,0.000000,36.333333,6.250000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
146,878740,NaN,NaN,2,0,0,0,10.000000,0.000000,0.000000,0.000000,83.333333,154.500000,139.333333,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0


In [22]:
X_cols = [col for col in df_nan.columns if col not in ['ID',  
                                                         'first_prediction', 'log_first_prediction'] ]

In [ ]:
scaler = MinMaxScaler()
X_cl = pd.DataFrame(scaler.fit_transform(df_nan[X_cols]), columns=df_nan[X_cols].columns)

with open('../../models/nan_classifier_w_user_data.pkl', 'rb') as file:
    nan_classifier = pickle.load(file)

nan_target_classes = nan_classifier.predict(X_cl)

df_nan.loc[:, 'target_class'] = nan_target_classes

df_nan_zero = df_nan[df_nan['target_class'] == 1]

df_nan_reg = df_nan[df_nan['target_class'] == 0]

df_nan_zero.loc[:, 'log_TARGET'] = -30.0
df_nan_zero.loc[:, 'TARGET'] = 0.0 

In [24]:
X_cols = [col for col in df_nan_reg.columns if col not in ['ID', 
                                                         'first_prediction', 'target_class', 'log_first_prediction'] ]

In [ ]:
scaler = MinMaxScaler()
X_re = pd.DataFrame(scaler.fit_transform(df_nan_reg[X_cols]), columns=df_nan_reg[X_cols].columns)

with open('../../models/nan_regressor_w_user_data.pkl', 'rb') as file:
    nan_regressor = pickle.load(file)

nan_reg_log_TARGET = nan_regressor.predict(X_re)

df_nan_reg.loc[:, 'log_TARGET'] = nan_reg_log_TARGET

df_nan_reg.loc[:, 'TARGET'] = np.power(np.e, df_nan_reg['log_TARGET']) - 1.0e-22

In [26]:
df_nan = pd.concat([df_nan_zero, df_nan_reg], ignore_index=True)
df_nan = df_nan.sort_values(by='ID')

In [27]:
df_nan.head()

,ID,first_prediction,log_first_prediction,Retention_count_D0_3,Retention_count_D4_7,Retention_count_D8_11,Retention_count_D12_15,Avg_Lvl_Count_D0_3,Avg_Lvl_Count_D4_7,Avg_Lvl_Count_D8_11,Avg_Lvl_Count_D12_15,Avg_Duration_Lvl0_3,Avg_Duration_Lvl4_7,Avg_Duration_Lvl8_10,Total_IAPRevenue,Total_AdRevenue,country_Brazil,country_Canada,country_Chile,country_Colombia,country_Ecuador,country_France,country_Germany,country_India,country_Indonesia,country_Italy,country_Mexico,country_Other,country_Peru,country_Spain,country_Turkey,country_United Kingdom,country_United States,country_Venezuela,platform_iOS,device_category_tablet,device_brand_Honor,device_brand_Huawei,device_brand_Motorola,device_brand_OPPO,device_brand_Other,device_brand_Samsung,device_brand_Vivo,device_brand_Xiaomi,has_ios_att_permission_True,ad_network_Nan,ad_network_Other,ad_network_applovin_int,ad_network_googleadwords_int,ad_network_unityads_int,target_class,log_TARGET,TARGET
0,878601,NaN,NaN,1,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,-30.000000,0.000000
1,878659,NaN,NaN,1,0,0,0,0.750000,0.000000,0.000000,0.000000,76.666667,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,-30.000000,0.000000
2,878680,NaN,NaN,1,0,0,0,3.000000,0.000000,0.000000,0.000000,64.000000,115.000000,131.333333,0.000000,0.000000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,-30.000000,0.000000
3,878710,NaN,NaN,1,1,0,0,1.000000,0.000000,0.000000,0.000000,36.333333,6.250000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,-30.000000,0.000000
4,878740,NaN,NaN,2,0,0,0,10.000000,0.000000,0.000000,0.000000,83.333333,154.500000,139.333333,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,-30.000000,0.000000


In [28]:
del(nan_regressor)
del(nan_classifier)

**df_test**

In [29]:
df_copy = df_test.copy()

In [30]:
economic_class_dummies = pd.get_dummies(df_copy.economic_class , prefix = 'economic_class', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, economic_class_dummies], axis = 1)

country_dummies = pd.get_dummies(df_copy.country , prefix = 'country', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, country_dummies], axis = 1)

platform_dummies = pd.get_dummies(df_copy.platform , prefix = 'platform', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, platform_dummies], axis = 1)

device_category_dummies = pd.get_dummies(df_copy.device_category , prefix = 'device_category', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, device_category_dummies], axis = 1)

device_brand_dummies = pd.get_dummies(df_copy.device_brand , prefix = 'device_brand', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, device_brand_dummies], axis = 1)

has_ios_att_permission_dummies = pd.get_dummies(df_copy.has_ios_att_permission , prefix = 'has_ios_att_permission', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, has_ios_att_permission_dummies], axis = 1)

ad_network_dummies = pd.get_dummies(df_copy.ad_network , prefix = 'ad_network', drop_first=True, dtype=int)
df_copy = pd.concat([df_copy, ad_network_dummies], axis = 1)

In [31]:
df_copy.drop(columns=['economic_class', 'country', 'platform', 'device_category', 'device_brand',
                     'has_ios_att_permission', 'ad_network'], inplace=True)

In [32]:
df_test = df_copy.copy()

In [33]:
df_test.head()

,ID,first_prediction,log_first_prediction,Retention_count_D0_3,Retention_count_D4_7,Retention_count_D8_11,Retention_count_D12_15,Avg_Lvl_Count_D0_3,Avg_Lvl_Count_D4_7,Avg_Lvl_Count_D8_11,Avg_Lvl_Count_D12_15,Avg_Duration_Lvl0_3,Avg_Duration_Lvl4_7,Avg_Duration_Lvl8_10,Total_IAPRevenue,Total_AdRevenue,economic_class_poor,economic_class_upper_middle_class,economic_class_wealthy,country_Brazil,country_Canada,country_Chile,country_Colombia,country_Ecuador,country_France,country_Germany,country_India,country_Indonesia,country_Italy,country_Mexico,country_Other,country_Peru,country_Spain,country_Turkey,country_United Kingdom,country_United States,country_Venezuela,platform_iOS,device_category_tablet,device_brand_Honor,device_brand_Huawei,device_brand_Motorola,device_brand_OPPO,device_brand_Other,device_brand_Samsung,device_brand_Vivo,device_brand_Xiaomi,has_ios_att_permission_True,ad_network_Nan,ad_network_Other,ad_network_applovin_int,ad_network_googleadwords_int,ad_network_unityads_int
0,878594,1.444805,0.367975,4,1,0,0,4.000000,0.000000,0.000000,0.000000,81.333333,164.750000,116.333333,0.000000,0.012716,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,878595,9.147972,2.213532,1,0,0,0,0.750000,0.000000,0.000000,0.000000,103.333333,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
2,878596,40.731158,3.706993,1,0,0,0,0.750000,0.000000,0.000000,0.000000,74.000000,0.000000,0.000000,0.000000,0.000000,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
3,878597,4.967959,1.603009,2,2,2,0,0.750000,4.250000,0.500000,0.000000,508.000000,695.750000,394.333333,0.000000,0.269068,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
4,878598,2.445842,0.894389,3,2,0,0,11.000000,8.750000,0.000000,0.000000,56.333333,85.250000,55.000000,0.000000,0.004202,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


In [34]:
del(df_copy)

In [35]:
X_cols = [col for col in df_test.columns if col not in ['ID', 'first_prediction'] ]

In [ ]:
scaler = MinMaxScaler()
X_cl = pd.DataFrame(scaler.fit_transform(df_test[X_cols]), columns=df_test[X_cols].columns)

with open('../../models/global_classifier_w_user_data.pkl', 'rb') as file:
    global_classifier = pickle.load(file)

global_target_classes = global_classifier.predict(X_cl)

df_test.loc[:, 'target_class'] = global_target_classes

df_test_zero = df_test[df_test['target_class'] == 1]

df_test_reg = df_test[df_test['target_class'] == 0]

df_test_zero.loc[:, 'log_TARGET'] = -30.0
df_test_zero.loc[:, 'TARGET'] = 0.0 

In [37]:
del(global_classifier)

In [38]:
X_cols = [col for col in df_test_reg.columns if col not in ['ID', 'first_prediction', 'target_class'] ]

In [ ]:
scaler = MinMaxScaler()
X_re = pd.DataFrame(scaler.fit_transform(df_test_reg[X_cols]), columns=df_test_reg[X_cols].columns)

with open('../../models/global_regressor_w_user_data.pkl', 'rb') as file:
    global_regressor = pickle.load(file)

global_reg_log_TARGET = global_regressor.predict(X_re)

df_test_reg.loc[:, 'log_TARGET'] = global_reg_log_TARGET

df_test_reg.loc[:, 'TARGET'] = np.power(np.e, df_test_reg['log_TARGET']) - 1.0e-22

In [40]:
df_test = pd.concat([df_test_zero, df_test_reg], ignore_index=True)
df_test = df_test.sort_values(by='ID')

In [ ]:
df_test.head()

In [41]:
del(global_regressor)

**df_sub**

In [42]:
df_sub = pd.concat([df_test, df_nan], ignore_index=True)

In [43]:
df_sub = df_sub.sort_values(by='ID')

In [44]:
df_sub['TARGET'] = df_sub['TARGET'] + (df_sub['Total_AdRevenue'] + df_sub['Total_IAPRevenue'])

In [45]:
df_sub = df_sub[['ID','TARGET']]

In [46]:
df_sub.to_csv('../../data/submissions/df_sub_w_user_data.csv', index=False)